In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
pd.options.display.max_columns = None
train_data = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
#train_data = train_data.dropna(axis = 1)
#also lets remove the output which is SalePrice in our case
y_train_data = train_data['SalePrice']
print(y_train_data)
train_data = train_data.drop(['SalePrice'], axis = 1)

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64


In [3]:
for col in train_data.columns:
    train_data[col] = train_data[col].fillna(train_data[col].mode()[0])

In [4]:
train_data.describe(include='object')

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
count,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460
unique,5,2,2,4,4,2,5,3,25,9,8,5,8,6,8,15,16,3,4,5,6,4,4,4,6,6,6,5,2,5,4,7,5,6,3,5,5,3,3,4,4,9,6
top,RL,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,TA,TA,PConc,TA,TA,No,Unf,Unf,GasA,Ex,Y,SBrkr,TA,Typ,Gd,Attchd,Unf,TA,TA,Y,Gd,MnPrv,Shed,WD,Normal
freq,1151,1454,1419,925,1311,1459,1052,1382,225,1260,1445,1220,726,1141,1434,515,504,1317,906,1282,647,686,1348,991,467,1294,1428,741,1365,1335,735,1360,1070,951,686,1392,1407,1340,1456,1336,1455,1267,1198


In [5]:
test_data = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [6]:
#Alroght as our data has both continuous and discrete values, lets, figure out if the categorical values have order or not
# I feel like all our categorical data doesnt have a natural ordering so lets use one hot encoding to encode this data
cat_cols = train_data.select_dtypes(include = ['object']).columns
encoded_train_data = pd.get_dummies(train_data, columns = cat_cols)
encoded_train_data = encoded_train_data.drop(['Id'], axis = 1)

In [7]:
#first I will standardize the data
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_data_std = scaler.fit_transform(encoded_train_data)
train_data_std

array([[ 0.07337496, -0.14618854, -0.20714171, ..., -0.11785113,
         0.4676514 , -0.30599503],
       [-0.87256276,  0.52499153, -0.09188637, ..., -0.11785113,
         0.4676514 , -0.30599503],
       [ 0.07337496, -0.01195252,  0.07347998, ..., -0.11785113,
         0.4676514 , -0.30599503],
       ...,
       [ 0.30985939, -0.1014432 , -0.14781027, ..., -0.11785113,
         0.4676514 , -0.30599503],
       [-0.87256276, -0.01195252, -0.08016039, ..., -0.11785113,
         0.4676514 , -0.30599503],
       [-0.87256276,  0.30126484, -0.05811155, ..., -0.11785113,
         0.4676514 , -0.30599503]])

In [8]:
#Now lets perform PCA
pca = PCA(n_components = 150)

pca.fit(train_data_std)
train_data_pca = pca.transform(train_data_std)

train_data_pca

array([[ 4.01601834e+00, -1.74947976e+00, -2.03872014e+00, ...,
         1.82238141e-01, -8.34584157e-02,  6.28563565e-02],
       [ 1.08089145e-01,  3.46748940e+00, -7.45594169e-01, ...,
        -2.41285990e+00,  3.59343872e-01, -8.70081112e-01],
       [ 4.78628830e+00, -7.42941338e-01, -1.30589022e+00, ...,
        -1.37937520e-01,  1.10796873e-01,  2.85513788e-01],
       ...,
       [ 8.68128900e-01, -1.09558264e+00,  2.64876433e+00, ...,
        -1.43619674e+00, -1.82017915e+00, -7.34080928e-01],
       [-3.13561629e+00,  3.30099184e+00, -1.04259253e+00, ...,
        -5.00861906e-03,  3.64594401e-01,  5.33355899e-01],
       [-1.26095111e+00,  3.82280920e+00, -1.46456431e+00, ...,
        -3.58665393e-03, -8.79416328e-01, -3.08125535e-01]])

In [9]:
#Now lets apply linear Regression
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

model = LinearRegression()
model.fit(train_data_pca, y_train_data)

LinearRegression()

In [10]:
test_data = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
test_data.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [11]:
#doing all the encoding,standardization, and cleansing for test_data also
for col in test_data.columns:
    test_data[col] = test_data[col].fillna(test_data[col].mode()[0])

cat_cols = test_data.select_dtypes(include = ['object']).columns
encoded_test_data = pd.get_dummies(test_data, columns = cat_cols)
encoded_test_data = encoded_test_data.drop('Id', axis = 1)

#columns that are not present in test data
cols_to_add = encoded_train_data.columns.difference(encoded_test_data.columns)
#creating the zero dataframe
zero_df = pd.DataFrame(np.zeros((len(encoded_test_data), len(cols_to_add))), columns = cols_to_add)

#concatinating B with the zeros
encoded_test_data = pd.concat([encoded_test_data, zero_df], axis = 1)

scaler = StandardScaler()
test_data_std = scaler.fit_transform(encoded_test_data)

pca = PCA(n_components = 150)

pca.fit(test_data_std)
test_data_pca = pca.transform(test_data_std)


In [12]:
cols_to_add

Index(['Condition2_RRAe', 'Condition2_RRAn', 'Condition2_RRNn',
       'Electrical_Mix', 'Exterior1st_ImStucc', 'Exterior1st_Stone',
       'Exterior2nd_Other', 'GarageQual_Ex', 'Heating_Floor', 'Heating_OthW',
       'HouseStyle_2.5Fin', 'MiscFeature_TenC', 'PoolQC_Fa',
       'RoofMatl_ClyTile', 'RoofMatl_Membran', 'RoofMatl_Metal',
       'RoofMatl_Roll', 'Utilities_NoSeWa'],
      dtype='object')

In [13]:
y_predict = model.predict(test_data_pca)

In [14]:
output = pd.DataFrame({'Id':test_data.Id, 'SalePrice':y_predict})
output.to_csv('submission.csv', index = False)